In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES']= '0'
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import numpy as np
import pretty_midi as pyd
from midi2audio import FluidSynth
import IPython.display as ipd
from arrangement_utils import *
import warnings
warnings.filterwarnings("ignore")

"""Download checkpoints and data at https://drive.google.com/drive/folders/17yB-Oae_4eGKJmqRS-LB8PwE2rqwZrUu?usp=sharing (455MB) and decompress at the directory"""
DATA_FILE_ROOT = '/data1/zhaojw/data_file_dir/'
DEVICE = 'cuda:0'
_, orchestrator, _, band_prompt = load_premise(DATA_FILE_ROOT, DEVICE, load_piano_arranger=False)

Loading piano to multi-track arrangement module (orchestrator). This may take 1 or 2 minutes ...
loading Slakh2100 Dataset ...


100%|██████████| 331/331 [00:02<00:00, 125.63it/s]


Initializing prior model
Finished.


### Initialize input and preference
We provide four sample piano arrangements for a quick inference. You should be able to directly run the code blocks after downloading the pre-trained checkpoints.

If you wish to test our model on your own lead sheet file, please initialize a sub-folder with its `SONG_NAME` in the `./demo` folder and put the file in, and name the file "arrangement_piano.mid". 

Please also specify `NOTE_SHIFT` (the number of pick-up beats if any).

In [ ]:
#SONG_NAME, NOTE_SHIFT, TEMPO = '07 The Moon Represents My Heart', 1.5, 70
#SONG_NAME, NOTE_SHIFT, TEMPO = '08 Suddenly Miss You So Bad', 22, 80
#SONG_NAME, NOTE_SHIFT, TEMPO = '09 I Only Care About You',18.5, 80
SONG_NAME, NOTE_SHIFT, TEMPO = '10 Floral Sea', 0, 75


melody, pno_red = read_piano_reduction('./demo', SONG_NAME, NOTE_SHIFT, melody_track_ID=0)
pno_red = pno_red.reshape((-1, 32, 128))

"""have a quick listen to the lead sheet"""
midi_path = f'./demo/{SONG_NAME}/arrangement_piano.mid'
audio_path = midi_path.replace('.mid', '.wav')
FluidSynth("/usr/share/sounds/sf2/FluidR3_GM.sf2", 16000).midi_to_audio(midi_path, audio_path)
ipd.Audio(audio_path, rate=16000)

### Orchestration

Stage 2: multi-track orchestration from piano

Firstly we're going to sample a piece from Slakh2100 dataset and utilize its instrument set for orchestration. 

You can choose which instruments you wish to have or not by passing thier program number to the `MUST_HAVE` or `MUSTNOT_HAVE` list. We support 34 instrument classes as shown [here](orchestrator/autoencoder_dataset.py).

In [21]:
MUST_HAVE = []
MUSTNOT_HAVE = [0] # 0: Acoustic Piano
func_prompt = prompt_sampling(pno_red, *band_prompt, MUSTNOT_HAVE=MUSTNOT_HAVE, DEVICE=DEVICE)

"""Set if use a 2-bar orchestral prompt for full-band arrangement (default True)""" 
USE_PROMPT = True

Prior model initialized with 10 tracks:
	['Organ', 'Pipe', 'Electric Piano', 'Choir and Voice', 'Clean Electric Guitar', 'Choir and Voice', 'Acoustic Guitar', 'Electric Bass', 'Clean Electric Guitar', 'Distorted Electric Guitar']


Now we orchestrate the piano arrangement from Stage 1 to a multi-track arrangement.

In [ ]:
USE_PROMPT = True
if USE_PROMPT:
    instruments, time_promt = func_prompt
else:
    instruments, _ = func_prompt
    time_promt = None
midi_collection = orchestration(pno_red, None, instruments, time_promt, orchestrator, DEVICE, blur=.25, p=.05, t=6, tempo=TEMPO, num_sample=4)

melody_track = matrix2midi(melody[np.newaxis, :, :], [82], init_tempo=TEMPO)
for idx, item in enumerate(midi_collection):
    item.instruments.append(melody_track.instruments[0])
    midi_path = f'./demo/{SONG_NAME}/arrangement_band-{str(idx).zfill(2)}.mid'
    item.write(midi_path)

"""have a quick listen to (one of) the orchestration results"""
audio_path = midi_path.replace('.mid', '.wav')
FluidSynth("/usr/share/sounds/sf2/FluidR3_GM.sf2", 16000).midi_to_audio(midi_path, audio_path)
ipd.Audio(audio_path, rate=16000)